In [9]:
import numpy as np
from sklearn import preprocessing

In [2]:
def encode(labels):
    return (labels -100)/300
def decode(enc_labels):
    return (labels*300)+100

In [3]:
number_to_train = 8000
number_to_test =2000

In [12]:
#features_fixed_nu_hits_orig = np.loadtxt("features_fixed_nu_hits2.csv")
#results_orig = np.loadtxt("results2.csv")
features_fixed_nu_hits_orig = np.load("features_fixed_nu_hits2_easy.npy")
results_orig = np.load("results2_easy.npy")

In [5]:
results_enc = encode(results_orig)
features_fixed_nu_hits = features_fixed_nu_hits_orig.reshape([10000,1000,3])
print results_enc.shape, features_fixed_nu_hits.shape
print features_fixed_nu_hits

(10000,) (10000, 1000, 3)
[[[  8.04076342e-01   5.14007839e+00  -2.89010094e+02]
  [ -5.20139323e+00  -7.85420461e-01   1.75958459e+02]
  [ -1.16234310e+00  -5.13203462e+00   4.96997006e+02]
  ..., 
  [ -1.55646975e+01   1.72426391e+01  -2.38978672e+02]
  [  2.18445338e+01  -8.18394347e+00   1.88000276e+02]
  [ -1.38551496e+01  -1.88109141e+01   7.60128975e+01]]

 [[  5.22592743e+00   3.67916895e-01  -2.27004249e+02]
  [ -6.25112915e-01  -5.21428249e+00   1.30283544e+01]
  [ -3.27319620e-01  -5.29447848e+00   1.12987782e+02]
  ..., 
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00]]

 [[  5.17388936e+00  -4.17068877e-01  -2.01980751e+02]
  [ -5.14501858e+00   1.42360817e+00  -2.13993046e+02]
  [ -6.38472762e-01  -5.43867831e+00   1.29739377e+01]
  ..., 
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
  [  0.0000

In [11]:
scaler = preprocessing.StandardScaler()
a = features_fixed_nu_hits

c = np.array([a[:,:,0].flatten(), a[:,:,1].flatten(), a[:,:,2].flatten()]).T  ## collect all x , y and z values of all hits
scaler.fit(c)

for i in range(a.shape[0]):
    tdata_contrib = scaler.transform(a[i,:,:])
    if i==0:
        transformed_data = [tdata_contrib]
    else:
        transformed_data = np.append(transformed_data, [tdata_contrib], axis=0)
        
    


assert features_fixed_nu_hits.shape == transformed_data.shape
features_fixed_nu_hits = transformed_data
print features_fixed_nu_hits

[[[  1.07736407e-01   7.04545081e-01  -1.88687349e+00]
  [ -7.13511304e-01  -1.07733433e-01   7.03658484e-01]
  [ -1.61171370e-01  -7.03575467e-01   2.49229691e+00]
  ..., 
  [ -2.13069273e+00   2.36358683e+00  -1.60812776e+00]
  [  2.98501808e+00  -1.12193687e+00   7.70748423e-01]
  [ -1.89691213e+00  -2.57870196e+00   1.46820455e-01]]

 [[  7.12424358e-01   5.03682184e-02  -1.54141330e+00]
  [ -8.77051637e-02  -7.14850160e-01  -2.04092465e-01]
  [ -4.69819431e-02  -7.25843576e-01   3.52822816e-01]
  ..., 
  [ -2.22099883e-03  -6.65207499e-05  -2.76678812e-01]
  [ -2.22099883e-03  -6.65207499e-05  -2.76678812e-01]
  [ -2.22099883e-03  -6.65207499e-05  -2.76678812e-01]]

 [[  7.05308153e-01  -5.72391059e-02  -1.40199705e+00]
  [ -7.05802074e-01   1.95084362e-01  -1.46892251e+00]
  [ -8.95321223e-02  -7.45610760e-01  -2.04395643e-01]
  ..., 
  [ -2.22099883e-03  -6.65207499e-05  -2.76678812e-01]
  [ -2.22099883e-03  -6.65207499e-05  -2.76678812e-01]
  [ -2.22099883e-03  -6.65207499e-05 

In [53]:
#fix number of hits
# order in ascending value of rho


from keras.layers import Input, Dense, Dropout, Flatten 
from keras.layers import Conv3D, MaxPooling3D
from keras.models import Model

# This returns a tensor
inputs = Input(shape=(1000,3,)) # (number of hits per event), (x,y,z)
#labels shape (,1)

x = Flatten()(inputs)



# a layer instance is callable on a tensor, and returns a tensor
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(1, activation='tanh')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='mse',   #mean squared error
              metrics=[])
model.fit(features_fixed_nu_hits[:number_to_train,:,:], results_enc[:number_to_train], epochs=2, batch_size=25)  # starts training




Epoch 1/2
8000/8000 [==============================] - 1s - loss: 0.1404     
Epoch 2/2
8000/8000 [==============================] - 1s - loss: 0.1125     


In [54]:
model.evaluate(features_fixed_nu_hits[number_to_train:number_to_train+number_to_test,:], results_enc[number_to_train:number_to_train+number_to_test], batch_size=25)

1325/2000 [==================>...........] - ETA: 0s

0.11084563294425606

predicted = model.predict(features_fixed_nu_hits[number_to_train:number_to_train+number_to_test,:])

In [38]:
print predicted, results_enc[number_to_train:number_to_train+number_to_test]

[[ 1.        ]
 [-1.        ]
 [ 0.00175916]
 ..., 
 [ 0.00175916]
 [ 1.        ]
 [-1.        ]] [ 0.00427553 -0.13653037 -0.08732112 ...,  0.02201658 -0.0313603
 -0.11749169]
